In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import functools
import json

In [41]:
def importing_json_files(file_wf):
    f_wf = open(file_wf) #informations for nf
    # returns JSON object as
    # a dictionary
    wf = json.load(f_wf)
    f_wf.close
    return wf

#importing the process info
proc_nf = importing_json_files('../json/nf_proc_shell_old.json')
proc_nf_tool = importing_json_files('../json/nf_proc_tool_shell.json')


In [5]:
proc_nf_tool[0]

{'name_process': 'MINIMAP2_ALIGN',
 'string_process': 'process MINIMAP2_ALIGN {\n    tag "$meta.id"\n    label \'process_medium\'\n\n    conda (params.enable_conda ? \'bioconda::minimap2=2.21 bioconda::samtools=1.12\' : null)\n    container "${ workflow.containerEngine == \'singularity\' && !task.ext.singularity_pull_docker_container ?\n        \'https://depot.galaxyproject.org/singularity/mulled-v2-66534bcbb7031a148b13e2ad42583020b9cd25c4:1679e915ddb9d6b4abda91880c4b48857d471bd8-0\' :\n        \'quay.io/biocontainers/mulled-v2-66534bcbb7031a148b13e2ad42583020b9cd25c4:1679e915ddb9d6b4abda91880c4b48857d471bd8-0\' }"\n\n    input:\n    tuple val(meta), path(reads)\n    path reference\n    val bam_format\n    val cigar_paf_format\n    val cigar_bam\n\n    output:\n    tuple val(meta), path("*.paf"), optional: true, emit: paf\n    tuple val(meta), path("*.bam"), optional: true, emit: bam\n    path "versions.yml"           , emit: versions\n\n    when:\n    task.ext.when == null || task.ext

In [42]:
#take out the tools not in the dicts
for proc in proc_nf_tool:
    tools = proc["tools"]
    tools_dict = []
    for el in proc["tools_dico"]:
        if el["name"] in tools:
            tools_dict.append(el)
    proc["tools_dico"] = tools_dict

In [10]:
proc_nf_tool[0]["tools_dico"][0]["topic"][0]

[{'uri': 'http://edamontology.org/topic_0102', 'term': 'Mapping'}]

In [44]:
topics_repart_nf = []
for proc in proc_nf_tool :
    topics = []
    for el in proc['tools_dico'] :
        for top in el["function"][0]["operation"]:
            if(len(top)>0):
                topics.append(top[0]["term"])
    topics=list(set(topics))
    topics_repart_nf.append(topics)
        

In [45]:
all_topics=[]
for el in topics_repart_nf:
    for top in el:
        all_topics.append(top)
all_topics = list(set(all_topics))

In [49]:
nb_use = []
for topic in all_topics:
    count = 0
    for liste in topics_repart_nf:
        if topic in liste:
            count+=1
    nb_use.append({"operation": topic, "count":count})

In [50]:
df_repart_topics=pd.DataFrame(nb_use)
df_repart_topics = df_repart_topics.sort_values(by="count", ascending=False)

In [51]:
df_repart_topics.head(20)

,operation,count
197,Data visualisation,1948
149,Editing,1935
225,Sequence analysis (general),1014
93,Polymorphism detection,978
182,Significance testing,816
49,Statistical calculation,816
232,Validation,731
172,Utility operation,720
85,Data handling,653
194,Sequence alignment generation,529
